In [1]:
import os
import pandas as pd
import numpy as np
import glob

In [9]:
sub = 'sub-04'
ses = 'ses-02'
battery = 'camcan'
task = 'finger-tapping'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob('*' + task + '*.txt'))
tsv_files = []
# for i in range(len(csv_files)):
i = 0
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [11]:
csv_files

['finger-tapping_sub-09_run-01_1618999400.txt',
 'finger-tapping_sub-09_run-02_1618999792.txt']

In [10]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_files[i]), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [11]:
run_start

1263.8859

In [12]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), header=1, sep=',', skipinitialspace=True)

In [13]:
# Find rows with trial information
trials = file_df['Block#'].str.contains('Rest')==False
# trials = file_df['Block#'] != 'Rest'

In [14]:
# Extract the onsets of stimulus, probe and response
trial_onsets = file_df.loc[trials]['StimulusOnset']

# The stimuli are always on screen for 1s
durations = np.ones(sum(trials))

# The trials are coded numerically, and there are 9 types.
# We are interested in 3 categories:
# “specified” trials, in which subjects are told which finger to press (type 1-4)
# “chosen” trials, in which subjects choose one of many fingers to press (type 5-8)
# null events, where no finger press is required (type 9)
types = file_df.loc[trials]['Stimulus#']
trial_types = np.array(np.zeros(sum(trials), dtype=np.object_))

# Find indices of 'specified' trials and mark as such
s_list = [1, 2, 3, 4]
for si, s in enumerate(s_list):
    trial_types[types == s] = 'specified'

# Find indices of 'chosen' trials and mark as such
c_list = [5, 6, 7, 8]
for ci, c in enumerate(c_list):
    trial_types[types == c] = 'chosen'

# Find indices of 'null' trials and mark as such
trial_types[types == 9] = 'null'

In [15]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)